<h1>Sample Data Factory</h1>
<br>
This notebook can be run while data is being collected to produce sample data. The notebook reads links.txt, articles.txt, and comments.text to bring togethor all of the required pieces. 

The end result will only contain complete records so the sample will only be as large as the smallest completed process.

In [56]:
import requests
import requests.auth
import time
import os
import json
import pandas as pd
import sys
import newspaper
import nltk

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Download the NLTK Stopwords Corpus
nltk.download()

from IPython.display import clear_output

In [7]:
def getToken(creds):
    client_auth = requests.auth.HTTPBasicAuth(creds['client_id'], creds['secret_id'])
    post_data = {"grant_type": "password", "username": creds['username'], "password": creds['pw']}
    headers = {"User-Agent": creds['user_agent']}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    if response.status_code == 200:
        print 'Credentials Verified: Token Recived'
    else:
        print 'Invalid Creds'

    auth = response.json()['token_type']+' '+response.json()['access_token']
    return {"Authorization": auth, "User-Agent": creds['user_agent']}

In [8]:
with open("creds.txt") as f:
    creds = dict([line.strip().split(':') for line in f])
token = getToken(creds)

Credentials Verified: Token Recived


In [14]:
links = []
#Tags to keep
tags= [ u'author',u'created_utc', u'domain', u'downs', u'gilded',u'is_self', u'likes', u'media', 'id',
 u'num_comments', u'num_reports', u'over_18', u'permalink',u'score', u'selftext', u'subreddit', u'thumbnail', u'title', u'ups', u'url']

with open('./links.txt') as data_file:
    for link in data_file:
        links.append(pd.read_json(link,orient='records',typ='series')[tags])
linkdf = pd.concat(links,axis=1).transpose()
print linkdf.shape
linkdf.head()

(29597, 20)


,author,created_utc,domain,downs,gilded,is_self,likes,media,id,num_comments,num_reports,over_18,permalink,score,selftext,subreddit,thumbnail,title,ups,url
0,trpftw,1.446582e+09,cnn.com,0,0,False,None,None,3recdd,3956,None,False,/r/science/comments/3recdd/mass_killings_and_s...,7376,,science,http://b.thumbs.redditmedia.com/JA2kgMdHb-dCuQ...,"Mass killings and school shootings spread ""con...",7376,http://www.cnn.com/2015/07/02/health/contagiou...
1,the_phet,1.447239e+09,ibtimes.co.uk,0,0,False,None,None,3se6lu,1074,None,False,/r/science/comments/3se6lu/algae_has_been_gene...,6687,,science,http://b.thumbs.redditmedia.com/y1CGKgl69hKw-s...,Algae has been genetically engineered to kill ...,6687,http://www.ibtimes.co.uk/algae-genetically-eng...
2,skoalbrother,1.448903e+09,phys.org,0,0,False,None,None,3uvg0o,2219,None,False,/r/science/comments/3uvg0o/researchers_find_ne...,6648,,science,http://b.thumbs.redditmedia.com/hZrhEdBoJp22oE...,"Researchers find new phase of carbon, make dia...",6648,http://phys.org/news/2015-11-phase-carbon-diam...
3,godsenfrik,1.447707e+09,news.stanford.edu,0,0,False,None,None,3t2exx,703,None,False,/r/science/comments/3t2exx/when_scientists_fal...,6252,,science,http://b.thumbs.redditmedia.com/FQPskQP8EejVh9...,"When scientists falsify data, they try to cove...",6252,http://news.stanford.edu/news/2015/november/fr...
4,avogadros_number,1.447108e+09,phys.org,0,0,False,None,None,3s6xe6,657,None,False,/r/science/comments/3s6xe6/dispersants_did_not...,6138,,science,http://b.thumbs.redditmedia.com/mFQzb4d2QNiyaE...,Dispersants did not help oil degrade in BP spi...,6138,http://phys.org/news/2015-11-dispersants-oil-d...


In [11]:
articles = []
with open('./article_data.txt') as data_file:
    for article in data_file:
        try:
            articles.append(pd.read_json(article,orient='records',typ='series'))
        except:
            print 'inlvalid JSON'
articledf = pd.concat(articles,axis=1).transpose()
print articledf.shape
articledf.head()

invalid JSON
(6165, 6)


,authors,keywords,publish_date,summary,text,url
0,"[Ben Smart, Special To Cnn]","{u'school': 1.01915708812, u'shootings': 1.026...",2015-07-02 00:00:00,"On average, mass shootings occur about once ev...",(CNN) Mass killings and school shootings sprea...,http://www.cnn.com/2015/07/02/health/contagiou...
1,"[Derek Keats Flickr, Hannah Osborne, Marc Cirera]","{u'toxic': 1.01308139535, u'cancer': 1.0261627...",2015-11-10 16:00:00+00:00,Algae has been genetically engineered to kill ...,Algae has been genetically engineered to kill ...,http://www.ibtimes.co.uk/algae-genetically-eng...
2,[],"{u'diamond': 1.04316546763, u'laser': 1.008633...",None,"If Q-carbon is harder than diamond, why would ...","This is a collection of 0.02, 0.03 and 0.04 ca...",http://phys.org/news/2015-11-phase-carbon-diam...
3,[Bjorn Carey],"{u'obfuscation': 1.01181102362, u'fraud': 1.01...",2015-11-16 00:00:00,Stanford researchers uncover patterns in how s...,Stanford researchers uncover patterns in how s...,http://news.stanford.edu/news/2015/november/fr...
4,[],"{u'oil': 1.05142857143, u'microbes': 1.0114285...",None,And that leads to more questions about where m...,"Samantha Joye, a professor of marine sciences ...",http://phys.org/news/2015-11-dispersants-oil-d...


In [53]:
comments = []
#Tags to keep
tags= [ u'author', u'body', u'body_html', u'controversiality', u'created', u'created_utc', u'distinguished', u'downs',
 u'edited', u'gilded', u'id', u'likes', u'link_id', u'name', u'num_reports', u'parent_id', u'replies', u'score',
 u'subreddit', u'ups']

with open('./comments.txt') as data_file:
    for comment in data_file:
        comments.append(pd.read_json(comment,orient='records',typ='series')[tags])
commentdf = pd.concat(comments,axis=1).transpose()

#Remove ['removed'] comments
commentdf = commentdf.loc[commentdf['body']!='[removed]']
commentdf.head()

,author,body,body_html,controversiality,created,created_utc,distinguished,downs,edited,gilded,id,likes,link_id,name,num_reports,parent_id,replies,score,subreddit,ups
0,straydog1980,There's a documented parallel with suicides th...,"&lt;div class=""md""&gt;&lt;p&gt;There&amp;#39;s...",0,1.446614e+09,1.446585e+09,None,0,False,0,cwnc95a,None,t3_3recdd,t1_cwnc95a,None,t3_3recdd,"{u'kind': u'Listing', u'data': {u'modhash': No...",2640,science,2640
2,MadBomber420,I had this thought years ago. Its one thing to...,"&lt;div class=""md""&gt;&lt;p&gt;I had this thou...",0,1.446612e+09,1.446583e+09,None,0,False,0,cwnapaa,None,t3_3recdd,t1_cwnapaa,None,t3_3recdd,"{u'kind': u'Listing', u'data': {u'modhash': No...",3389,science,3389
5,satin_pajamas,There's an interesting New Yorker article abou...,"&lt;div class=""md""&gt;&lt;p&gt;There&amp;#39;s...",0,1.446621e+09,1.446592e+09,None,0,False,0,cwnhadl,None,t3_3recdd,t1_cwnhadl,None,t3_3recdd,"{u'kind': u'Listing', u'data': {u'modhash': No...",226,science,226
6,trpftw,"&gt;""What we believe may be happening is natio...","&lt;div class=""md""&gt;&lt;blockquote&gt;\n&lt;...",0,1.446611e+09,1.446582e+09,None,0,1.446587e+09,0,cwna17j,None,t3_3recdd,t1_cwna17j,None,t3_3recdd,"{u'kind': u'Listing', u'data': {u'modhash': No...",533,science,533
9,PainMatrix,The conclusions from the [actual open-access a...,"&lt;div class=""md""&gt;&lt;p&gt;The conclusions...",0,1.446613e+09,1.446585e+09,None,0,1.446588e+09,0,cwnbrhw,None,t3_3recdd,t1_cwnbrhw,None,t3_3recdd,"{u'kind': u'Listing', u'data': {u'modhash': No...",209,science,209


In [84]:
#Subset For Complete Records
#Remove links with no articles
linkdf = linkdf.loc[linkdf.url.isin(articledf.url)]
#Remove links with no comments
commentdf['pid'] = commentdf.parent_id.apply(lambda x: str.split(str(x),'_')[1])
complete_sample_links = linkdf.loc[linkdf['id'].isin(commentdf['pid'])]

#Join Link Features and Article Features
#Join on PID opposed to URL!!!
complete_sample_articles = articledf.merge(complete_sample_links, on='url')

complete_sample_comments = commentdf[commentdf.pid.isin(complete_sample_links['id'])]
print str(complete_sample_articles.shape[0])+' Sample Articles'
print str(complete_sample_comments.shape[0])+ ' Sample Comments'


3430 Sample Articles
106041 Sample Comments


In [68]:
#Create Comment Features
#########################
#Tokenized Comments
stopset = set(stopwords.words('english'))
def getTokens(text):
    tokens=word_tokenize(str(text))
    return [w for w in tokens if not w in stopset]

complete_sample_comments['tokens'] = complete_sample_comments.body.apply(tokenize)

#Comment Length
complete_sample_comments['comment_length'] = complete_sample_comments.body.apply(len)

#Number of Words
complete_sample_comments['n_tokens'] = complete_sample_comments.tokens.apply(len)

#Comment Keywords
complete_sample_comments['keywords'] = complete_sample_comments.body.apply(newspaper.nlp.keywords)

print complete_sample_comments.columns

Index([          u'author',             u'body',        u'body_html',
       u'controversiality',          u'created',      u'created_utc',
          u'distinguished',            u'downs',           u'edited',
                 u'gilded',               u'id',            u'likes',
                u'link_id',             u'name',      u'num_reports',
              u'parent_id',          u'replies',            u'score',
              u'subreddit',              u'ups',              u'pid',
                 u'tokens',   u'comment_length',         u'n_tokens',
               u'keywords'],
      dtype='object')


In [85]:
#Create Article Features
#########################
#Tokens
complete_sample_articles['tokens'] = complete_sample_articles.text.apply(tokenize)

#Article Length
complete_sample_articles['article_len'] = complete_sample_articles.text.apply(len)

#Number of Words
complete_sample_articles['n_tokens'] = complete_sample_articles.tokens.apply(len)

complete_sample_articles.columns

Index([     u'authors',     u'keywords', u'publish_date',      u'summary',
               u'text',          u'url',       u'tokens',  u'article_len',
           u'n_tokens',       u'author',  u'created_utc',       u'domain',
              u'downs',       u'gilded',      u'is_self',        u'likes',
              u'media',           u'id', u'num_comments',  u'num_reports',
            u'over_18',    u'permalink',        u'score',     u'selftext',
          u'subreddit',    u'thumbnail',        u'title',          u'ups'],
      dtype='object')

In [73]:
#TODO
#Similarity Features 
############################


#Similarity between each comment features and article features




In [86]:
#Write Samole Article and Comment dfs to csv
complete_sample_comments.to_csv('./sample_training_comments.csv',sep=',',encoding='utf-8',index=False)
complete_sample_articles.to_csv('./sample_training_articles.csv',sep=',',encoding='utf-8',index=False)

In [88]:
#Load Sample Article Comment dfs from csv
comments = pd.read_csv('./sample_training_comments.csv',encoding='utf-8')
articles = pd.read_csv('./sample_training_articles.csv',encoding='utf-8')

print comments.shape
print articles.shape

(106080, 21)
(3430, 28)
